In [1]:
import time
import os

import scanpy as sc
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import models.loadImg as loadImg
import models.modelsCNN as modelsCNN
import models.optimizer as optimizer

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import scipy.stats

import statsmodels.stats.multitest as multitest
import networkx as nx

import anndata as ad

In [2]:
#load model
os.environ["CUDA_LAUNCH_BLOCKING"] = "2" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
use_cuda=True
datadir='/media/xinyi/dcis2idc/data'
name='exp0'
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name
plottype='umap'
sampledir=plotsavepath
savedir=os.path.join(sampledir,'embedding_'+plottype)
clustersavedir=os.path.join(sampledir,'cluster_reordered')
clustersavedir_old=os.path.join(sampledir,'cluster')

with open(os.path.join(datadir,'processed','train_cnnvae_names'), 'rb') as input:
    allImgNames=pickle.load(input)


br1003aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR1003a specs.xlsx',header=10)
br301Specs=pd.read_excel('/media/xinyi/dcis2idc/data/BR301 specs.xlsx',header=10)
br8018aSpecs=pd.read_excel('/media/xinyi/dcis2idc/data/BR8018a specs.xlsx',header=10)
br1003aSpecs.index=br1003aSpecs.loc[:,'Position']
br301Specs.index=br301Specs.loc[:,'Position']
br8018aSpecs.index=br8018aSpecs.loc[:,'Position']
progList=np.copy(allImgNames)
for s in np.unique(allImgNames):
    ssplit=s.split('_')
    if 'br1003a'==ssplit[0]:
        prog_s=br1003aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br301'==ssplit[0]:
        prog_s=br301Specs.loc[(ssplit[-1],'Pathology diagnosis')]
    elif 'br8018a'==ssplit[0]:
        prog_s=br8018aSpecs.loc[(ssplit[-1],'Pathology diagnosis')]
    progList[allImgNames==s]=prog_s
    
progNames,progCounts=np.unique(progList,return_counts=True)
progSampleRate={}
for p in range(progNames.size):
    progSampleRate[progNames[p]]=np.min(progCounts)/progCounts[p]
    
ep=311
np.random.seed(6)
# plotPCT=4500
plottingIdx_i=np.array([])
n_pcs=50
uniqueImgNames,imgNameIdx=np.unique(allImgNames,return_index=True)
for i in range(1):
    for sidx in range(uniqueImgNames.size):
        s=uniqueImgNames[sidx]
        p=progList[imgNameIdx[sidx]]
        print(s+' '+p)
        nsamples=int(np.sum(allImgNames==s)*progSampleRate[p])
        plottingIdx_i=np.concatenate((plottingIdx_i,
                                    np.random.choice(np.arange(allImgNames.shape[0])[allImgNames==s],nsamples,replace=False)))
    
ncluster=8
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(ep)+'_plottingIdx_progBalanced_'+str(i)
with open(os.path.join(clustersavedir_old,savenamecluster+'_reordered'), 'rb') as output:
    clusterRes=pickle.load(output)
    
with open(os.path.join(clustersavedir_old,savenamecluster), 'rb') as output:
    clusterRes_old=pickle.load(output)
    
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels'), 'rb') as output:
    cellIDlist=pickle.load(output)
    
#load NMCO
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)

allstats=None
alllabels=None
alllabels_old=None
allvarnames=None
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames)[sidx]

    path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'nmco_features',s.split('_')[-1] +'.csv')
    if not os.path.exists(path_s):
        print('DNE '+path_s)
    print(s)
    plottingIdx_i_s=plottingIdx_i.astype(int)[allImgNames[plottingIdx_i.astype(int)]==s]-nameIdx[sidx]
    assert np.min(plottingIdx_i_s)>=0

    stats_s=pd.read_csv(path_s)
    stats_s.index=stats_s.loc[:,'label']
    stats_s=stats_s.loc[cellIDlist[s][plottingIdx_i_s]].to_numpy()[:,2:-2]
    print(stats_s.shape)


#         ssplit=s.split('_')
    slabels=clusterRes[allImgNames[plottingIdx_i.astype(int)]==s]
    slabels_old=clusterRes_old[allImgNames[plottingIdx_i.astype(int)]==s]

    if allstats is None:
        allstats=stats_s
        alllabels=np.copy(slabels)
        alllabels_old=np.copy(slabels_old)
    else:
        allstats=np.concatenate((allstats,stats_s),axis=0)
        alllabels=np.concatenate((alllabels,np.copy(slabels)))
        alllabels_old=np.concatenate((alllabels_old,slabels_old))


br1003a_1_cytokeratin_555_aSMA_647_hoechst_A1 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A4 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A5 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A6 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A7 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A8 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A9 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C10 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C2 Hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6 Atypical hyperplasia
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C7 Hyperplasia
br1003a_1_cytokeratin_555_

br301_6_collagen1_647_hoechst_D2 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_D3 Invasive ductal carcinoma and breast tissue
br301_6_collagen1_647_hoechst_D4 Invasive ductal carcinoma
br301_6_collagen1_647_hoechst_D5 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_D6 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E1 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E2 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E3 Invasive ductal carcinoma and breast tissue
br301_6_collagen1_647_hoechst_E4 Invasive ductal carcinoma
br301_6_collagen1_647_hoechst_E5 Ductal carcinoma in situ with early infiltratio
br301_6_collagen1_647_hoechst_E6 Ductal carcinoma in situ with early infiltratio
br8018a_1_cytokeratin_555_aSMA_647_hoechst_A1 Invasive ductal carcinoma
br8018a_1_cytokeratin_555_aSMA_647_hoechst_A10 Invasive ductal carcinoma
br8018a_1_cytoke

(361, 201)
br1003a_3_collagen1_647_hoechst_A7
(716, 201)
br1003a_3_collagen1_647_hoechst_A8
(481, 201)
br1003a_3_collagen1_647_hoechst_A9
(274, 201)
br1003a_3_collagen1_647_hoechst_C1
(2720, 201)
br1003a_3_collagen1_647_hoechst_C10
(779, 201)
br1003a_3_collagen1_647_hoechst_C2
(1286, 201)
br1003a_3_collagen1_647_hoechst_C3
(1172, 201)
br1003a_3_collagen1_647_hoechst_C4
(2233, 201)
br1003a_3_collagen1_647_hoechst_C5
(1350, 201)
br1003a_3_collagen1_647_hoechst_C6
(1136, 201)
br1003a_3_collagen1_647_hoechst_C7
(890, 201)
br1003a_3_collagen1_647_hoechst_C8
(556, 201)
br1003a_3_collagen1_647_hoechst_C9
(824, 201)
br1003a_3_collagen1_647_hoechst_I1
(1003, 201)
br1003a_3_collagen1_647_hoechst_I10
(1564, 201)
br1003a_3_collagen1_647_hoechst_I2
(1201, 201)
br1003a_3_collagen1_647_hoechst_I3
(1379, 201)
br1003a_3_collagen1_647_hoechst_I7
(1300, 201)
br1003a_3_collagen1_647_hoechst_I8
(1609, 201)
br1003a_3_collagen1_647_hoechst_I9
(853, 201)
br1003a_4_cytokeratin_555_gh2ax_647_hoechst_A1
(253, 20

(7843, 201)
br8018a_3_collagen1_647_hoechst_B7
(351, 201)
br8018a_3_collagen1_647_hoechst_B8
(230, 201)
br8018a_3_collagen1_647_hoechst_B9
(339, 201)
br8018a_3_collagen1_647_hoechst_F10
(467, 201)
br8018a_3_collagen1_647_hoechst_F3
(202, 201)
br8018a_3_collagen1_647_hoechst_F4
(362, 201)
br8018a_3_collagen1_647_hoechst_F6
(297, 201)
br8018a_3_collagen1_647_hoechst_F7
(341, 201)
br8018a_3_collagen1_647_hoechst_F8
(476, 201)
br8018a_3_collagen1_647_hoechst_F9
(419, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A2
(259, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A3
(453, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A5
(150, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A7
(488, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A8
(211, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B4
(484, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B5
(276, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B6
(8272, 201)
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B7
(356, 201)
br80

In [3]:
seed=3
epochs=6000
saveFreq=200
lr=0.001 #initial learning rate
weight_decay=0 #Weight for L2 loss on embedding matrix.

# batchsize=4
batchsize=8000
model_str='fc3'

kernel_size=4
stride=2
padding=1

fc_dim1=128
fc_dim2=128
fc_dim3=128


dropout=0.01
kl_weight=0.0000001

name='exp0_clusterClf_nmco_'+savenamecluster+'fcl3'
logsavepath='/media/xinyi/dcis2idc/log/cnnvae'+name
modelsavepath='/media/xinyi/dcis2idc/models/cnnvae'+name
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name

# Create model
torch.manual_seed(seed)
nclasses=np.unique(alllabels).size
if use_cuda:
    torch.cuda.manual_seed(seed)
    
if model_str=='fc3':
    model = modelsCNN.FC_l3(allstats.shape[1],fc_dim1,fc_dim2,fc_dim3,nclasses,0.5,regrs=False)
if model_str=='fc5':
    model = modelsCNN.FC_l5(allstats.shape[1],fc_dim1,fc_dim2,fc_dim3,fc_dim4,fc_dim5,nclasses,0.5,regrs=False)
if model_str=='fc1':
    model = modelsCNN.FC_l1(allstats.shape[1],fc_dim1,nclasses,regrs=False)
if model_str=='fc0':
    model = modelsCNN.FC_l0(allstats.shape[1],nclasses,regrs=False)

if use_cuda:
    model.cuda()
    
model.load_state_dict(torch.load(os.path.join(modelsavepath,str(5800)+'.pt')))



In [4]:
model.eval()
allstats=allstats.astype(float)
allstats=scipy.stats.zscore(allstats,axis=0,nan_policy='omit')
allstats=np.nan_to_num(allstats,nan=0)

In [9]:
trainIdx=np.arange(allstats.shape[0])
allgrad_byLabel=np.zeros_like(allstats)
for i in range(int(np.ceil(trainIdx.shape[0]/batchsize))):
    trainIdx_i=trainIdx[i*batchsize:min((i+1)*batchsize,trainIdx.shape[0])]

#         trainInput=trainInputnp[trainIdx]
#         labels=trainLabelsnp[trainIdx]
    if use_cuda:
        trainInput=torch.tensor(allstats[trainIdx_i]).cuda().float()
    trainInput.requires_grad = True
    model.zero_grad()
    pred = model(trainInput)
    
    for gidx in range(trainIdx_i.size):
        grad=torch.autograd.grad(pred[gidx,alllabels_old[trainIdx_i[gidx]]],trainInput,retain_graph=True)[0][gidx]
        allgrad_byLabel[trainIdx_i[gidx]]=grad.cpu().numpy()

In [5]:
stats_s=pd.read_csv(path_s)
nmcoNames=np.array(stats_s.columns[2:-2])

In [11]:
gradsavepath=os.path.join(plotsavepath,'grad_reordered')
if not os.path.exists(gradsavepath):
    os.mkdir(gradsavepath)

In [12]:
gradAvg=np.zeros((np.unique(alllabels).size,nmcoNames.size))
for l in np.unique(alllabels):
    gradAvg[l]=np.mean(allgrad_byLabel[alllabels==l],axis=0)

In [13]:
fig, ax = plt.subplots(figsize=(100, 10))
im = ax.imshow(gradAvg,cmap='seismic',vmin=-np.max(np.abs(gradAvg)),vmax=np.max(np.abs(gradAvg)))
ax.set_xticks(np.arange(nmcoNames.size))
ax.set_xticklabels(nmcoNames)
ax.set_yticks(np.arange(np.unique(alllabels).size))
ax.set_yticklabels(np.unique(alllabels))
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
#     fig.set_figheight(35)
#     fig.set_figwidth(35)
fig.colorbar(im)
fig.tight_layout()
plt.savefig(os.path.join(gradsavepath,'all.jpg'))
plt.close()

In [ ]:
nmcoSigIdx=np.array([])
stats={}
pvals=np.array([])
pval_startIdx=[]
currIdx=0
for l in np.unique(alllabels):
    stats[l]=pd.DataFrame({'avg':np.mean(allgrad_byLabel[alllabels==l],axis=0),
                       'std':np.std(allgrad_byLabel[alllabels==l],axis=0),
                       'fc':(np.mean(allgrad_byLabel[alllabels==l],axis=0)/np.mean(allgrad_byLabel[alllabels!=l],axis=0))},
                      index=nmcoNames)
    pvalues=scipy.stats.ttest_ind(allgrad_byLabel[alllabels==l],allgrad_byLabel[alllabels!=l],equal_var=False)[1]
    stats[l]['pval']=pvalues
    pvals=np.concatenate((pvals,pvalues))
    pval_startIdx.append(currIdx)
    currIdx+=pvalues.size

pval_corr=multitest.fdrcorrection(pvals,0.05,'indep')[1]

for l in np.unique(alllabels):
    stats[l]['pval_corr']=pval_corr[pval_startIdx[l]:(pval_startIdx[l]+stats[l]['pval'].size)]
    stats[l].to_csv(os.path.join(gradsavepath,str(l)+'_stats.csv'))
    nmcoSigIdx=np.concatenate((nmcoSigIdx,np.arange(stats[l].shape[0])[np.logical_and(np.logical_or(stats[l]['fc']<0.8,stats[l]['fc']>1.2),stats[l]['pval_corr']<0.05)]))

nmcoSigIdx=np.unique(nmcoSigIdx).astype(int)
fig, ax = plt.subplots(figsize=(100, 10))
im = ax.imshow(gradAvg[:,nmcoSigIdx],cmap='seismic',vmin=-np.max(np.abs(gradAvg[:,nmcoSigIdx])),vmax=np.max(np.abs(gradAvg[:,nmcoSigIdx])))
ax.set_xticks(np.arange(nmcoSigIdx.size))
ax.set_xticklabels(nmcoNames[nmcoSigIdx])
ax.set_yticks(np.arange(np.unique(alllabels).size))
ax.set_yticklabels(np.unique(alllabels))
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
#     fig.set_figheight(35)
#     fig.set_figwidth(35)
fig.colorbar(im)
fig.tight_layout()
plt.savefig(os.path.join(gradsavepath,'all_pval05_fc1.2.jpg'))
plt.close()

In [15]:
df = pd.DataFrame(allgrad_byLabel[:,nmcoSigIdx],columns=nmcoNames[nmcoSigIdx])
corr = df.corr()
# keep only upper triangle elements (excluding diagonal elements)
mask_keep = np.triu(np.ones(corr.shape), k=1).astype('bool').reshape(corr.size)
# melt (unpivot) the dataframe and apply mask
sr = corr.stack()[mask_keep]
# filter and get names
edges = sr[sr > 0.8].reset_index().values[:, :2]
g = nx.from_edgelist(edges)
ls_cc = []
for cc in nx.connected_components(g):
    ls_cc.extend(list(cc))
for n in nmcoNames[nmcoSigIdx]:
    if n not in ls_cc:
        ls_cc.append(n)
        
df_plot=pd.DataFrame(gradAvg[:,nmcoSigIdx],columns=nmcoNames[nmcoSigIdx])
df_plot=df_plot[ls_cc]

fig, ax = plt.subplots(figsize=(100, 10))
im = ax.imshow(df_plot.to_numpy(),cmap='seismic',vmin=-np.max(np.abs(gradAvg[:,nmcoSigIdx])),vmax=np.max(np.abs(gradAvg[:,nmcoSigIdx])))
ax.set_xticks(np.arange(nmcoSigIdx.size))
ax.set_xticklabels(ls_cc)
ax.set_yticks(np.arange(np.unique(alllabels).size))
ax.set_yticklabels(np.unique(alllabels))
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
#     fig.set_figheight(35)
#     fig.set_figwidth(35)
fig.colorbar(im)
fig.tight_layout()
plt.savefig(os.path.join(gradsavepath,'all_pval05_fc1.2_ordered.jpg'))
plt.close()

In [16]:
nmcoSigIdx.size

201

In [7]:
#also test NMCO difference directly
nmcodesavepath=os.path.join(plotsavepath,'nmcode_reordered')
if not os.path.exists(nmcodesavepath):
    os.mkdir(nmcodesavepath)

In [8]:
nmcoAvg=np.zeros((np.unique(alllabels).size,nmcoNames.size))
for l in np.unique(alllabels):
    nmcoAvg[l]=np.mean(allstats[alllabels==l],axis=0)
    
fig, ax = plt.subplots(figsize=(100, 10))
im = ax.imshow(nmcoAvg,cmap='seismic',vmin=-np.max(np.abs(nmcoAvg)),vmax=np.max(np.abs(nmcoAvg)))
ax.set_xticks(np.arange(nmcoNames.size))
ax.set_xticklabels(nmcoNames)
ax.set_yticks(np.arange(np.unique(alllabels).size))
ax.set_yticklabels(np.unique(alllabels))
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
#     fig.set_figheight(35)
#     fig.set_figwidth(35)
fig.colorbar(im)
fig.tight_layout()
plt.savefig(os.path.join(nmcodesavepath,'all.jpg'))
plt.close()

In [9]:
nonzeroIdx=np.sum(allstats==0,axis=0)==0
allstats_nonzero=allstats[:,nonzeroIdx]
    
nmcoSigIdx=np.array([])
stats={}
pvals=np.array([])
pval_startIdx=[]
currIdx=0
for l in np.unique(alllabels):
    stats[l]=pd.DataFrame({'avg':np.mean(allstats_nonzero[alllabels==l],axis=0),
                       'std':np.std(allstats_nonzero[alllabels==l],axis=0),
                       'fc':(np.mean(allstats_nonzero[alllabels==l],axis=0)/np.mean(allstats_nonzero[alllabels!=l],axis=0))},
                      index=nmcoNames[nonzeroIdx])
    pvalues=scipy.stats.ttest_ind(allstats_nonzero[alllabels==l],allstats_nonzero[alllabels!=l],equal_var=False)[1]
    stats[l]['pval']=pvalues
    pvals=np.concatenate((pvals,pvalues))
    pval_startIdx.append(currIdx)
    currIdx+=pvalues.size

pval_corr=multitest.fdrcorrection(pvals,0.01,'indep')[1]

for l in np.unique(alllabels):
    stats[l]['pval_corr']=pval_corr[pval_startIdx[l]:(pval_startIdx[l]+stats[l]['pval'].size)]
    stats[l].to_csv(os.path.join(nmcodesavepath,str(l)+'_stats01.csv'))
    nmcoSigIdx=np.concatenate((nmcoSigIdx,np.arange(stats[l].shape[0])[np.logical_and(np.abs(stats[l]['avg'])>0.5,np.logical_and(stats[l]['pval_corr']<0.01,np.logical_or(stats[l]['fc']<0.8,stats[l]['fc']>1.2)))]))

nmcoSigIdx=np.unique(nmcoSigIdx).astype(int)
fig, ax = plt.subplots(figsize=(100, 10))
im = ax.imshow(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx],cmap='seismic',vmin=-np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])),vmax=np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])))
ax.set_xticks(np.arange(nmcoSigIdx.size))
ax.set_xticklabels(nmcoNames[nonzeroIdx][nmcoSigIdx])
ax.set_yticks(np.arange(np.unique(alllabels).size))
ax.set_yticklabels(np.unique(alllabels))
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
#     fig.set_figheight(35)
#     fig.set_figwidth(35)
fig.colorbar(im)
fig.tight_layout()
plt.savefig(os.path.join(nmcodesavepath,'all_pval01_fc1.2_avg05.jpg'))
plt.close()

In [10]:
df = pd.DataFrame(allstats_nonzero[:,nmcoSigIdx],columns=nmcoNames[nonzeroIdx][nmcoSigIdx])
corr = df.corr()
# keep only upper triangle elements (excluding diagonal elements)
mask_keep = np.triu(np.ones(corr.shape), k=1).astype('bool').reshape(corr.size)
# melt (unpivot) the dataframe and apply mask
sr = corr.stack()[mask_keep]
# filter and get names
edges = sr[sr > 0.8].reset_index().values[:, :2]
g = nx.from_edgelist(edges)
ls_cc = []
for cc in nx.connected_components(g):
    ls_cc.extend(list(cc))
for n in nmcoNames[nonzeroIdx][nmcoSigIdx]:
    if n not in ls_cc:
        ls_cc.append(n)
        
df_plot=pd.DataFrame(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx],columns=nmcoNames[nonzeroIdx][nmcoSigIdx])
df_plot=df_plot[ls_cc]

fig, ax = plt.subplots(figsize=(100, 10))
im = ax.imshow(df_plot.to_numpy(),cmap='seismic',vmin=-np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])),vmax=np.max(np.abs(nmcoAvg[:,nonzeroIdx][:,nmcoSigIdx])))
ax.set_xticks(np.arange(nmcoSigIdx.size))
ax.set_xticklabels(ls_cc)
ax.set_yticks(np.arange(np.unique(alllabels).size))
ax.set_yticklabels(np.unique(alllabels))
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
#     fig.set_figheight(35)
#     fig.set_figwidth(35)
fig.colorbar(im)
fig.tight_layout()
plt.savefig(os.path.join(nmcodesavepath,'all_pval01_fc1.2_avg05_ordered.jpg'))
plt.close()

In [11]:
nmcoSigIdx.size

117

In [16]:
len(ls_cc)

9

In [15]:
ls_cc

[['weighted_moments-0-0',
  'weighted_moments-0-1',
  'weighted_moments_central-2-2',
  'moments-3-0',
  'convex_area',
  'minor_axis_length',
  'weighted_centroid-0',
  'moments-2-3',
  'moments-3-1',
  'max_radius',
  'moments-2-0',
  'moments_central-0-2',
  'weighted_moments-2-0',
  'moments_central-2-0',
  'med_radius',
  'feret_max',
  'weighted_moments-1-2',
  'weighted_moments-3-1',
  'sum_posi_curv',
  'weighted_moments-0-3',
  'weighted_moments-2-2',
  'min_calliper',
  'moments-1-0',
  'centroid-1',
  'weighted_moments-1-3',
  'weighted_moments_central-0-0',
  'weighted_moments-1-1',
  'perimeter',
  'weighted_moments-2-1',
  'len_posi_curv',
  'moments-1-1',
  'moments-0-3',
  'weighted_moments-0-2',
  'equivalent_diameter',
  'moments-1-2',
  'major_axis_length',
  'weighted_moments_central-2-0',
  'correlation_1',
  'moments-0-1',
  'weighted_moments-3-3',
  'centroid-0',
  'moments-0-0',
  'moments_central-2-2',
  'd25_radius',
  'avg_radius',
  'moments_central-0-0',
  

In [14]:
#draw cc on pseudotime
df = pd.DataFrame(allstats_nonzero[:,nmcoSigIdx],columns=nmcoNames[nonzeroIdx][nmcoSigIdx])
corr = df.corr()
# keep only upper triangle elements (excluding diagonal elements)
mask_keep = np.triu(np.ones(corr.shape), k=1).astype('bool').reshape(corr.size)
# melt (unpivot) the dataframe and apply mask
sr = corr.stack()[mask_keep]
# filter and get names
edges = sr[sr > 0.8].reset_index().values[:, :2]
g = nx.from_edgelist(edges)
ls_cc = []
for cc in nx.connected_components(g):
    ls_cc.append(list(cc))

In [17]:
neworder=[1, 5, 3, 7, 2, 0, 4, 6]
#use chosen subcluster number and save plots
sc.settings.verbosity = 3
# subcluster=8
subclusterDict={0:[4],1:[6],2:[8],3:[6],4:[6],5:[6],6:[6],7:[4]}
ncluster=8

plotepoch=311
plotsavenameAdd='_plottingIdx_progBalanced_'+str(0)+'_reordered'
savenamecluster0='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
kmeans_sub=(np.zeros(clusterRes.size)-1).astype(str)
for c in np.unique(clusterRes):
    print('cluster'+str(c))
    for subcluster in subclusterDict[neworder[c]]:
        subclustersavedir=os.path.join(clustersavedir_old,savenamecluster0+'_plottingIdx_progBalanced_'+str(0)+'_subcluster'+str(neworder[c]))
#         latent_adata=ad.AnnData(latent[plottingIdx_i.astype(int)][clusterRes==c])

        savenamecluster='minibatchkmean_ncluster'+str(subcluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_plottingIdx_progBalanced_'+str(0)
        with open(os.path.join(subclustersavedir,savenamecluster), 'rb') as output:
            subclusterRes=pickle.load(output)

        kmeans_sub[clusterRes==c]=np.char.add(np.repeat(str(c)+'-',subclusterRes.size),subclusterRes.astype(str))


alllabels_sub=np.array([])
allidx=np.array([])
idx_temp=np.arange(plottingIdx_i.size)
for sidx in range(uniquenames.size):
    s=np.unique(allImgNames)[sidx]

    path_s=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'nmco_features',s.split('_')[-1] +'.csv')
    if not os.path.exists(path_s):
        print('DNE '+path_s)
    print(s)
    plottingIdx_i_s=plottingIdx_i.astype(int)[allImgNames[plottingIdx_i.astype(int)]==s]-nameIdx[sidx]
    assert np.min(plottingIdx_i_s)>=0

    sidx=idx_temp[allImgNames[plottingIdx_i.astype(int)]==s]
    slabels_sub=kmeans_sub[allImgNames[plottingIdx_i.astype(int)]==s]

    alllabels_sub=np.concatenate((alllabels_sub,slabels_sub))
    allidx=np.concatenate((allidx,sidx))

cluster0
cluster1
cluster2
cluster3
cluster4
cluster5
cluster6
cluster7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A4
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A5
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A6
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_A9
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C10
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C2
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C3
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C4
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C5
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C6
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C7
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C8
br1003a_1_cytokeratin_555_aSMA_647_hoechst_C9
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I1
br1003a_1_cytokeratin_555_aSMA_647_hoechst_I10
br1003

br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A3
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A5
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A7
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_A8
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B4
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B5
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B6
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_B7
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F3
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F4
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F7
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_F8
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_H1
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_H3
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_H5
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_H6
br8018a_4_cytokeratin_555_gh2ax_647_hoechst_H8


In [18]:
with open(os.path.join(datadir,'processed','latent311'), 'rb') as input:
    latent=pickle.load(input)
    
latent_adata=ad.AnnData(latent[plottingIdx_i.astype(int)])
latent_adata.obs['kmeans_sub']=kmeans_sub
latent_adata.obs['kmeans_sub']=latent_adata.obs['kmeans_sub'].astype('category')
latent_adata.obs['kmeans']=clusterRes.astype(str)
latent_adata.obs['kmeans']=latent_adata.obs['kmeans'].astype('category')

<ipython-input-18-9877bc29d4e7>:4: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  latent_adata=ad.AnnData(latent[plottingIdx_i.astype(int)])


In [20]:
with open(os.path.join('/media/xinyi/dcis2idc/plots/cnnvaeexp0','cluster_reordered','minibatchkmean_ncluster'+str(8)+'n_pcs'+str(50)+'epoch'+str(ep)+'_plottingIdx_progBalanced_'+str(0)+'_reordered'+'_subcluster_all','pagaPos_reordered'), 'rb') as output:
    trainPagaPos=pickle.load(output)

In [28]:
#add single cell plots & use all cells -- use previously calculated graph in combined subcluster (neighbors=4)
#thresh=1

# nneighbors_list=[4,10] 
np.random.seed(3)
sc.tl.pca(latent_adata, svd_solver='arpack')
sc.pp.neighbors(latent_adata, n_neighbors=4, n_pcs=20)
sc.tl.paga(latent_adata, groups='kmeans_sub')
plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams['figure.dpi'] = 1200
sc.settings.figdir=nmcodesavepath

vmax=None
# for ccidx in range(len(ls_cc)):
for ccidx in [2]:
    if ccidx==2:
        vmax=4.5
    cc=ls_cc[ccidx]
#     np.savetxt(os.path.join(gradsavepath,'_cc'+str(ccidx)+'.txt'),cc,fmt='%s',delimiter=',')
    
    v='nmco_cc'+str(ccidx)
    protvalues=np.repeat(np.nan,latent_adata.shape[0])
    protvalues[allidx.astype(int)]=np.mean(df[np.array(cc)].to_numpy(),axis=1)
    latent_adata.obs[v]=protvalues

    sc.pl.paga(latent_adata, color=[v],save='graph'+v+'_allcells.pdf',pos=trainPagaPos,fontsize=18,fontoutline=1,max_edge_width=10, node_size_scale=5,node_size_power=1,threshold=0.01,show=False,random_state=6)
    sc.tl.umap(latent_adata, init_pos='paga')
    sc.pl.umap(latent_adata,color=[v],legend_loc='on data',save='graph_umap'+v+'.pdf',show=False,vmax=vmax)



#     sc.pl.paga(latent_adata, color=[v],solid_edges='connectivities_tree',
#         dashed_edges='connectivities',layout='rt_circular',
#         node_size_scale=0.5,
#         node_size_power=0.9,
#         max_edge_width=0.7,
#         fontsize=10,
#         save='tree'+v+'_allcells.png',show=False)
#     sc.tl.umap(latent_adata, init_pos='paga')
#     sc.pl.umap(latent_adata,color=[v],legend_loc='on data',save='tree_umap'+v+'.png',show=False)

--> added 'pos', the PAGA positions (adata.uns['paga'])
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:43)


In [29]:
sc.pl.umap(latent_adata,color=[v],legend_loc='on data',save='graph_umap'+v+'.pdf',show=False,vmax=4.5)

<AxesSubplot:title={'center':'nmco_cc2'}, xlabel='UMAP1', ylabel='UMAP2'>

In [25]:
ccidx=2
cc=ls_cc[ccidx]
#     np.savetxt(os.path.join(gradsavepath,'_cc'+str(ccidx)+'.txt'),cc,fmt='%s',delimiter=',')

v='nmco_cc'+str(ccidx)
protvalues=np.repeat(np.nan,latent_adata.shape[0])
protvalues[allidx.astype(int)]=np.mean(df[np.array(cc)].to_numpy(),axis=1)
plt.hist(protvalues,bins=20)

(array([2.3745e+04, 8.2351e+04, 6.6527e+04, 3.7382e+04, 2.1588e+04,
        1.3304e+04, 8.4950e+03, 5.6600e+03, 3.4020e+03, 1.9150e+03,
        1.0050e+03, 4.9100e+02, 2.4200e+02, 1.4100e+02, 5.8000e+01,
        3.3000e+01, 1.3000e+01, 4.0000e+00, 3.0000e+00, 5.0000e+00]),
 array([-1.25275062, -0.82006211, -0.38737361,  0.0453149 ,  0.47800341,
         0.91069191,  1.34338042,  1.77606893,  2.20875744,  2.64144594,
         3.07413445,  3.50682296,  3.93951146,  4.37219997,  4.80488848,
         5.23757699,  5.67026549,  6.102954  ,  6.53564251,  6.96833101,
         7.40101952]),
 <BarContainer object of 20 artists>)